In [0]:
dbutils.widgets.dropdown(name = 'environment', choices= ['dev', 'uat', 'prod'], defaultValue = 'dev', label = 'select your environment')
environment = dbutils.widgets.get('environment')

In [0]:
%run "./04_common"

In [0]:
from pyspark.sql.functions import when
def add_road_category_name(df):
    print('Adding new column Road_Category_Name')
    processed_df = df.withColumn('Road_Category_Name', when(df.Road_Category =='TA', 'Class A Trunk Road')
                                            .when(df.Road_Category =='TM', 'Class A Trunk Motor')
                                            .when(df.Road_Category =='PA', 'Class A Principal Road')
                                            .when(df.Road_Category =='PM', 'Class A Principal Motorway')
                                            .when(df.Road_Category =='M', 'Class B Road')
                                            .otherwise('NA'))
    print('OK')
    return processed_df

def add_road_type(df):
    print('Adding new column Road_Type')
    processed_df = df.withColumn('Road_Type', when(df.Road_Category_Name.contains('Class A'), 'Major')
                                             .when(df.Road_Category_Name.contains('Class B'), 'Minor')
                                             .otherwise('NA'))
    print('OK')
    return processed_df

In [0]:
def write_silver_roads(df,environment):
    df.writeStream\
            .format('delta')\
            .option('checkpointLocation', checkpoint_url + 'Silver_Roads_load/checkpoint')\
            .option('path', silver_url + 'silver_roads/') \
            .trigger(availableNow=True)\
            .outputMode('append')\
            .toTable(f'traffic_{environment}_catalog.{db_name}.silver_roads')
    print('successfully write bronze roads to silver table')

In [0]:
df_bronze_roads = spark.readStream.table(f'traffic_{environment}_catalog.{db_name}.raw_roads')

In [0]:
processed_df1 = check_duplicates(df_bronze_roads)
processed_df2 = check_null(processed_df1, processed_df1.schema.names)

In [0]:
processed_df3 = add_road_category_name(processed_df2)
processed_df4 = add_road_type(processed_df3)

In [0]:
write_silver_roads(processed_df4,environment)